сначала просто посмотрим как у нас уменьшается функция ошибок:

In [1]:
import numpy as np
import sys
sys.path.insert(0, "./Optimization-lib")
from lrs import *
from gradient_descent import GradientDecent, ScipyWrapper
from newton import NewtonCG
from function_wrapper import FunctionWrapper
from output import *
from graphics_plotter import GraphicsPlotter
from stochastic_gradient_descent import StochGradientDecent
from ucimlrepo import fetch_ucirepo

wine_quality = fetch_ucirepo(id=186)

X = wine_quality.data.features  # Признаки
y = wine_quality.data.targets

X = X.values
y = y.values.ravel()

def normalize(X_train):
    # 1. Вычисляем среднее и стандартное отклонение ТОЛЬКО на тренировочных данных
    mean = X_train.mean(axis=0)  # Среднее по каждому признаку
    std = X_train.std(axis=0)     # Стандартное отклонение по каждому признаку
    std[std == 0] = 1            # Избегаем деления на 0 (если std=0)

    # 2. Нормализуем тренировочные и тестовые данные
    X_train = (X_train - mean) / std
    # X_test = (X_test - mean) / std

    return X_train

X = normalize(X)
print(X)
print("...")
print(y)

def generate_weight_bounds(X, abs_bound: int) -> list:
    """
    Генерирует границы весов для всех параметров модели (включая intercept)
    n_features: Количество признаков (без учёта const)
    """
    n_features = X.shape[1]
    return [[-1 * abs_bound, abs_bound] for _ in range(n_features + 1)]  # +1 для intercept

def generate_start(X) -> list:
    n_features = X.shape[1]
    return [1 for _ in range(n_features + 1)]  # +1 для intercept


bounds = generate_weight_bounds(X, 100)
start = generate_start(X)


gradient.clear()
sgd = StochGradientDecent(exponential_decay(0.01, 0.0001), generate_weight_bounds(X, 1000), X, y, 5)
error_min = sgd.find_min(generate_start(X), 100)
print(error_min)

pretty_dataset_print(sgd, "EXP1", error_min, gradient)


/########################################################################


# добавить график изменения значения error_min от количества итераций (?)
#


[[ 0.14247327  2.18883292 -2.19283252 ...  1.81308951  0.19309677
  -0.91546416]
 [ 0.45103572  3.28223494 -2.19283252 ... -0.11507303  0.99957862
  -0.58006813]
 [ 0.45103572  2.55330026 -1.91755268 ...  0.25811972  0.79795816
  -0.58006813]
 ...
 [-0.55179227 -0.6054167  -0.88525328 ... -1.42124765 -0.47897144
  -0.91546416]
 [-1.32319841 -0.30169391 -0.12823371 ...  0.75571005 -1.016626
   1.9354021 ]
 [-0.93749534 -0.78765037  0.42232597 ...  0.25811972 -1.41986693
   1.09691202]]
...
[5 5 5 ... 6 7 6]
1.1897035216105487

        EXP1
            found min error:         1.189704
            found weights:      ['5.17575224846130854672', '0.23221107728766987588', '0.08473465432080465320', '0.05804017386406113560', '0.39784568165554545782', '0.12804677345021930890', '0.20685906818722618605', '0.11694056862290568011', '-0.06194505261198809953', '0.18325565384914738187', '-0.07865386711855928970', '0.75290825047093712374']
            steps count:          100
            function cal

на каждом шаге мы ровно один раз считаем градиент (по формуле) и 1 раз функцию ошибок. Так что количество вызова функций = кол-ву шагов. Кажется можно выкинуть эти параметры из print_dataset_pretty.
(Ps - можно считать градиент старым способом - будет сильно дольше (пару строк раскоментить и посмотреть сколько вызовов функции... -> сказать что мы молодцы и сразу градиет по формулу считаем. Так же с регулярностью - просто ее производную прибоаляем к градиету))

сейчас хотим выделять из dataset набор данных для обучения, и набор данных для проверки достоверности подобранных коэффициентов.



In [2]:
def custom_train_test_split(X, y, test_size=0.2, random_state=None):
    """просто рандомно делим на массивы"""
    if random_state is not None:
        np.random.seed(random_state)

    indices = np.arange(len(X))
    np.random.shuffle(indices)

    split_idx = int(len(X) * (1 - test_size))

    X_train = X[indices[:split_idx]]
    X_test = X[indices[split_idx:]]
    y_train = y[indices[:split_idx]]
    y_test = y[indices[split_idx:]]

    return X_train, X_test, y_train, y_test


X_train, X_test, y_train, y_test = custom_train_test_split(X, y, test_size=0.2, random_state=42)

bounds = generate_weight_bounds(X, 100)
start = generate_start(X)


gradient.clear()
sgd = StochGradientDecent(exponential_decay(0.01, 0.0001), generate_weight_bounds(X_train, 1000), X_train, y_train, 5)
error_min = sgd.find_min(generate_start(X_train), 120) # !! полезно менять max_steps
pretty_dataset_print(sgd, "EXP1", error_min, gradient)

# Получаем обученные веса
trained_weights = sgd.get_x()

# Прогнозируем на тестовой выборке
X_test = np.c_[np.ones(X_test.shape[0]), X_test] # это мы добавили столбец 1 что бы отдельно с константами не возиться (1, x1, x2...)
y_pred = np.dot(X_test, trained_weights)  # предсказание результата


# Вычисляем метрики качества на тестовой выборке
# сюда вскяие функции для анализа впихнуть и вывод написать что мы молодцы

print("\nРезультаты на тестовой выборке:")
print("True values:", y_test)
print("Predicted values:", y_pred)
#
def mean_squared_error(y_true, y_pred):
    return ((y_true - y_pred) ** 2).mean()

def mean_absolute_error(y_true, y_pred):
    return np.abs(y_true - y_pred).mean()

mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
# r2 = r2_score(y_test, y_pred)

print(f"\nMSE: {mse:.4f}")
print(f"MAE: {mae:.4f}")
# print(f"R2 Score: {r2:.4f}")




        EXP1
            found min error:         1.140617
            found weights:      ['5.28782849775108854118', '0.25866594877269261099', '0.14873552789412053388', '0.15811103632418255227', '0.22299977638661713786', '-0.04987182617298462767', '0.14806890090945951099', '0.24801519411139943960', '-0.01072839543144898933', '0.37947050117061664753', '0.11656715171751827476', '0.72660682683487420874']
            steps count:          120
            function calls count: 5761
            gradient calls count: 240
            hessian calls count:  0
          

Результаты на тестовой выборке:
True values: [7 7 5 ... 5 5 5]
Predicted values: [6.19866864 6.63806819 5.75763635 ... 4.85995832 5.1101066  4.96203293]

MSE: 1.1602
MAE: 0.8363
